In [68]:
import pandas as pd

In [69]:
import numpy as np

In [70]:
from scipy import sparse

In [71]:
DAMPING_FACTOR = 0.15

In [106]:
data = pd.read_csv("soc-sign-bitcoinotc.csv", sep=',', names=["SOURCE", "TARGET", "RATING", "TIME"], index_col="RATING", usecols=["SOURCE", "TARGET", "RATING"])

In [107]:
data.sort_index(ascending=False, inplace=True)

In [108]:
data = data.ix[10:8]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [109]:
data.sort_values(by=["SOURCE", "TARGET"], inplace=True)

data.set_index("SOURCE", drop=True, append=False, inplace=True, verify_integrity=False)

In [110]:
data.reset_index(drop=True, inplace=True)

In [111]:
sources = data["SOURCE"].tolist()
targets = data["TARGET"].tolist()
n = max( max(sources, key=int), max(targets, key=int)) + 1

In [112]:
values = list()

for source in sources:
        values.append(1/len(data[data.SOURCE == source]))

In [113]:
matrix_a = sparse.coo_matrix((values,(sources,targets)), shape=(n,n)).toarray()

In [114]:
matrix_b = np.full( (n, n), (DAMPING_FACTOR/n))

In [136]:
matrix_m = (( 1 - DAMPING_FACTOR) * matrix_a ) + ( DAMPING_FACTOR * matrix_b)

In [137]:
matrix_v = np.full( (n, n), (1/n))

In [128]:
count = 0
def pagerank( matrix_m, matrix_v) :
    global count
    count += 1
    if (sum(abs(matrix_m.dot(matrix_v) - matrix_v)) > 0.001) :
        return pagerank( matrix_m, matrix_m.dot(matrix_v))
    else :
        return (matrix_m.dot(matrix_v))

In [138]:
count = 0
def pagerank(v) :
    global count
    count += 1
    if (sum(abs((matrix_m*v)-v)) > 0.001) :
        return pagerank(matrix_m*v, matrix_m)
    else :
        return matrix_m*v

In [139]:
result = pagerank( matrix_v)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()